In [2]:
from transformers import AutoImageProcessor, AutoModel
import torch

c:\Users\HP\Desktop\E-Vision-Projects\Merchandiser_Products_Counter\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = 'facebook/dinov2-large'

In [4]:
processor = AutoImageProcessor.from_pretrained(model_name)
model  = AutoModel.from_pretrained(model_name)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\HP\Desktop\E-Vision-Projects\Merchandiser_Products_Counter\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--facebook--dinov2-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 1 files: 100

In [11]:
def get_embedding(image):
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        print(outputs)
        # Use CLS token embedding
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding.numpy()

In [12]:
from PIL import Image

In [13]:
image = '../data/test_output/cropped_image_2.jpg'
image = Image.open(image)

In [14]:
embedding = get_embedding(image)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 2.7682, -0.7520,  0.8486,  ...,  0.0129, -2.0608, -0.4539],
         [ 2.1516,  0.8273, -0.7372,  ...,  0.3455, -1.5367,  0.2737],
         [ 1.5849,  0.7777, -0.0699,  ..., -0.3213, -2.6683,  1.2853],
         ...,
         [ 0.5084, -0.4084,  0.0531,  ...,  0.8579, -0.3337, -0.0351],
         [ 0.5830, -1.3127, -0.1754,  ..., -0.6313, -0.1508,  0.0484],
         [ 1.8436, -1.8138, -0.3123,  ..., -0.6692,  0.0078,  0.4187]]]), pooler_output=tensor([[ 2.7682, -0.7520,  0.8486,  ...,  0.0129, -2.0608, -0.4539]]), hidden_states=None, attentions=None)


In [10]:
embedding

array([ 2.768203  , -0.75202346,  0.84858453, ...,  0.01292449,
       -2.0608444 , -0.45385855], shape=(1024,), dtype=float32)